In [1]:
import sqlalchemy as sql
import psycopg2
import pandas as pd
import numpy as np

In [2]:
#Useful commands for connecting to postgres
### db_string = ###

In [3]:
#Helper functions
def write_to_sql(data, target_table, overwrite_option):
    '''Function to write to sql.
    
    Input: data - dataframe containing the data to be written to SQL, target_table - string containing the name of
           table in the database, overwrite_option - the options for the if_exits argument in pandas.to_sql. 
           replace = get rid of the values in the database, append = append the values on to the existing one
           
    Output: None'''
    #You need to change all the column names to lower case because SQL is stupid
    data.columns = data.columns.str.lower()
    data.dropna(axis = 0, how = 'any', inplace = True)
    engine = sql.create_engine(db_string)
    with engine.connect() as conn:
        data.to_sql(target_table, con = conn, if_exists = overwrite_option, index = False, method = 'multi')
        conn.closed
    engine.dispose()
    
def read_from_sql(table):
    '''Function that takes all of the results from a sql table
    
    Input: table - table data is extracted from
    
    Output: A dataframe containing the results'''
    engine = sql.create_engine(db_string)
    query = 'SELECT * FROM {}'.format(table)
    with engine.connect() as conn:
        df = pd.read_sql_query(query, conn)
    engine.dispose()
    return df

def make_series(target, fight_series):
    '''Function to turn the fight series values into features that are stored in the database.
    
    Input: target - the column of fighters that is going to be made into a new series,
           fight_series - the dataframe containing the fight data.
           
    Output: A dataframe with a fighter's fights that have been stripped of their opponent.'''
    fight_series.sort_values(by = [target,"date"], ascending = [True, True], axis = 0, inplace = True)
    fight_series.reset_index(drop = True, inplace = True)
    fight_series["winner_boolean"] = (fight_series[target] == fight_series["winner"])*1
    output = fight_series[[target, "win_by", "last_round", "date", "winner_boolean"]]
    output.rename({target:"fighter"}, axis = 1,  inplace = True)
    return output

def combine_series(red, blue):
    '''Function to combine and clean all of the fighter's fights into one combined and cleaned dataset.
    
    Input: red - a dataframe containing the fighter and pertinent fight data, blue - a dataframe containing 
                 the fighter and pertinent fight data.
                 
                 List of columns in both dataframes: fighter, win_by, last_round, date, and winner_boolean.
           
    Output: A dataframe of the combined data that has been cleaned and sorted.'''
    merged = red.append(blue)
    merged['win_by_categorical'] = merged[['win_by', 'winner_boolean']].apply(classify, axis = 1)
    merged.sort_values(by = ['fighter','date'], ascending = [True, False], axis = 0, inplace = True)
    merged['winner_boolean'] = merged['winner_boolean'].astype('bool')
    return merged

def classify(x):
    '''Function to classify all possible outcomes to the categorical variable.
    
    Input: row of a dataframe containing both the win by method as a string 
           and a boolean denoting whether or not the favorite won.
           
    Output: int of the corresponding category.
    '''
    if x[0] == 'Submission' and x[1] == 0:
        return 1
    elif x[0] in ['Decision - Unanimous', 'Unanimous Decision'] and x[1] == False:
        return 2
    elif x[0] in ['Decision - Split','Split Decision'] and x[1] == False:
        return 3
    elif x[0] in ['TKO - Doctor\'s Stoppage', 'KO', 'TKO', 'KO/TKO', 'Could Not Continue'] and x[1] == False:
        return 4
    elif x[0] in ['Decision - Majority','Majority Decision'] and x[1] == False:
        return 5
    elif x[0] in ['DQ', 'Disqualification', 'Overturned', 'Forfeit','No Contest', 'Other'] and x[1] == False:
        return 6
    elif x[0] == 'Submission' and x[1] == True:
        return 7
    elif x[0] in ['Decision - Unanimous', 'Unanimous Decision'] and x[1] == True:
        return 8
    elif x[0] in ['Decision - Split','Split Decision'] and x[1] == True:
        return 9
    elif x[0] in ['TKO - Doctor\'s Stoppage', 'KO', 'TKO', 'KO/TKO', 'Could Not Continue'] and x[1] == True:
        return 10
    elif x[0] in ['Decision - Majority','Majority Decision'] and x[1] == True:
        return 11
    elif x[0] in ['DQ', 'Disqualification', 'Overturned','Forfeit','No Contest', 'Other'] and x[1] == True:
        return 12
    
def remove_duplicates(fight_series):
    '''A function meant to deal with duplicate fights. The drop duplicates method in the pandas class 
    does not account for duplicates in other columns so this function is meant to check for duplicate fights when
    the fighter names are in different columns.
    
    Input : A data frame that contain the fight data.
    
    Output: a data frame that contains fight data that has been checked for duplicates in both columns'''
    counter = 0
    print("Number of fights pre-clean: {}".format(fight_series.shape[0]))
    while counter < fight_series.shape[0]:
        fight_series.iloc[counter, 0], fight_series.iloc[counter, 1] = fight_series.iloc[counter, 1], fight_series.iloc[counter, 0]
        fight_series.drop_duplicates(subset = ["r_fighter", "b_fighter", "date"], keep = "first", inplace = True)
        fight_series.iloc[counter, 0], fight_series.iloc[counter, 1] = fight_series.iloc[counter, 1], fight_series.iloc[counter, 0]
        fight_series.reset_index(drop = True, inplace = True)
        if counter % 1000 == 0:
            print("Iteration No. {}".format(counter+1))
        counter+=1
    print("Number of fights post-clean: {}".format(fight_series.shape[0]))
    return fight_series

In [4]:
#UFC Data clean
UFC = pd.read_csv("raw_total_fight_data.csv", delimiter = ";")
UFC = UFC[["R_fighter", "B_fighter", "win_by", "last_round", "date", "Winner"]]
UFC.columns = UFC.columns.str.lower()
UFC["date"]= pd.to_datetime(UFC["date"], format = "%B %d, %Y")
UFC["winner_boolean"] = UFC["winner"] == UFC["r_fighter"]
UFC['result'] = UFC[['win_by','winner_boolean']].apply(classify, axis = 1)
UFC.dropna(axis = 0, how = 'any', inplace = True)

#Scraped Data clean
Sherdog = pd.read_csv("scraped_fights_v2.csv")
Sherdog["Date"] = pd.to_datetime(Sherdog["Date"], format = '%b / %d / %Y')
Sherdog["Method"] = Sherdog["Method"].str.replace("[(-].+", "")
Sherdog["Method"] = Sherdog["Method"].str.strip()
Sherdog.drop(Sherdog[Sherdog.Method.isin(["Rich Mitchell", "132091", "", "<br/>", "su", "DG"])].index, inplace = True)
Sherdog[~Sherdog.Fighter.str.contains("Born:")]
Sherdog[~Sherdog.Opponent.str.contains("Born:")]
fix = {"Desqualification":"Disqualification",
      "Decision unanimous": "Unanimous Decision",
      "Unanimous decision": "Unanimous Decision",
      "Submision":"Submission",
      "Submisson":"Submission",
      "ubmission":"Submission",
      "Technical Submission": "Submission",
      "ee Stoppage from a Cut)":"No Contest",
      "NC":"No Contest",
      "Guillotine Choke":"Submission",
      "Drew":"Draw",
      "Technical Submission":"Submission",
      "Verbal Submission":"Submission",
      "Towel":"Forfeit",
      "Tko":"TKO",
      "K.O":"KO",
      "ND":"No Contest",
      "Technical Draw":"Draw",
      "Technical Decision":"Split Decision",
      "Decision":"Split Decision",
      "Decison":"Split Decision",
      "Majority decision":"Majority Decision",
      "Techinal Submission":"Submission",
      "No Decision":"No Contest",
      "Injury Default":"No Contest",
      "Injury":"No Contest",
      "Majority Draw":"Draw",
      "ТКО":"TKO",
      "ΤΚΟ":"TKO",
      "DRAW":"Draw",
      "Split Draw":"Draw", 
      "DQ":"Disqualification"}
Sherdog.Method = Sherdog.Method.replace(fix)
Sherdog = Sherdog[Sherdog.Method.notna()]
Sherdog.drop("Time", axis = 1)
Sherdog.drop(Sherdog[Sherdog["Method"] == "Draw"].index, inplace = True)
Sherdog["Result_boolean"] = Sherdog["Result"] == 'win'
Sherdog['win_by_categorical'] = Sherdog[['Method', 'Result_boolean']].apply(classify, axis = 1)

In [5]:
#Transformation that takes combines data checks for duplicates
#Standardize the columns between the two datasets
Sherdog["Winner"] = np.select(condlist=[Sherdog["Result"]=="win", Sherdog["Result"]!="win"],
                             choicelist=[Sherdog["Fighter"], Sherdog["Opponent"]])
Sherdog = Sherdog[["Fighter","Opponent","Method","Rounds","Date","Winner","Result_boolean","win_by_categorical"]]
Sherdog.columns = ["r_fighter","b_fighter","win_by","last_round","date","winner","winner_boolean","result"]
#Combine them and reset the index
merged = Sherdog.append(UFC)
#Resetting the index is important, without it the drop duplicates function will not function properly
merged.reset_index(drop = True, inplace = True)
merged.drop_duplicates(subset= ["r_fighter","b_fighter","date"], keep = False, inplace = True)
#Call the remove duplictaes function. The issue here is that some fights are duplicates but the fighter and opponent
#columns have been flipped. The pandas drop duplicates function will not catch it so we have to call our own
fight_series = remove_duplicates(merged)

Number of fights pre-clean: 55663
Iteration No. 1
Iteration No. 1001
Iteration No. 2001
Iteration No. 3001
Iteration No. 4001
Iteration No. 5001
Iteration No. 6001
Iteration No. 7001
Iteration No. 8001
Iteration No. 9001
Iteration No. 10001
Iteration No. 11001
Iteration No. 12001
Iteration No. 13001
Iteration No. 14001
Iteration No. 15001
Iteration No. 16001
Iteration No. 17001
Iteration No. 18001
Iteration No. 19001
Iteration No. 20001
Iteration No. 21001
Iteration No. 22001
Iteration No. 23001
Iteration No. 24001
Iteration No. 25001
Iteration No. 26001
Iteration No. 27001
Iteration No. 28001
Iteration No. 29001
Iteration No. 30001
Iteration No. 31001
Iteration No. 32001
Iteration No. 33001
Iteration No. 34001
Iteration No. 35001
Iteration No. 36001
Iteration No. 37001
Iteration No. 38001
Iteration No. 39001
Iteration No. 40001
Iteration No. 41001
Iteration No. 42001
Iteration No. 43001
Iteration No. 44001
Iteration No. 45001
Iteration No. 46001
Iteration No. 47001
Iteration No. 48001

In [8]:
#Writing to the database
write_to_sql(fight_series, "t_fight_series", "replace")
#Strip out the individual fighter information to be used as features going forward
fighter_series = combine_series(make_series("r_fighter", fight_series), make_series("b_fighter", fight_series))
write_to_sql(fighter_series, "t_fighter_series", "replace")

/Users/mikehenry/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
#Double check to make sure everything was written correctly to the database
test = read_from_sql("t_fight_series")
print(test.shape)
test.head()

(54616, 8)


,r_fighter,b_fighter,win_by,last_round,date,winner,winner_boolean,result
0,Yaroslav Amosov,Mark Lemminger,TKO,1,2020-08-21,Yaroslav Amosov,True,10
1,Yaroslav Amosov,David Rickels,Submission,2,2019-08-24,Yaroslav Amosov,True,7
2,Yaroslav Amosov,Gerald Harris,Split Decision,3,2018-07-13,Yaroslav Amosov,True,9
3,Yaroslav Amosov,Diogo Cavalcanti,Submission,1,2017-03-18,Yaroslav Amosov,True,7
4,Yaroslav Amosov,Khasanbek Abdulaev,TKO,2,2016-03-03,Yaroslav Amosov,True,10


In [10]:
test = read_from_sql("t_fighter_series")
print(test.shape)
test.head()

(109232, 6)


,fighter,win_by,last_round,date,winner_boolean,win_by_categorical
0,\n Born...,TKO,2,2018-08-17,False,4
1,\n Born...,Split Decision,3,2018-01-26,True,9
2,\n Born...,Split Decision,3,2017-07-14,False,3
3,\n Born...,TKO,1,2016-10-21,False,4
4,\n Born...,Split Decision,3,2016-06-17,False,3
